In [1]:
import pptk
from color_dict import COLOR_DICT
import glob
import re
from PIL import Image 
import numpy as np

In [2]:
# This function will invoke a pptk viewer to render the points 
# Param: data: an array of points coordinate. Each point's coordinate should has the format of [x, y, z]
#       pointSize: the size of the point to be rendered on the screen
# Return: none

def displayPoints(data, color, pointSize):
    v = pptk.viewer(data)
    v.attributes(color)
    v.set(point_size=pointSize)

In [3]:
organData = [np.empty((1, 3)) for x in range(len(COLOR_DICT))]
organColor = [np.empty((1, 3)) for x in range(len(COLOR_DICT))]
pngPath = ".\masksToView\*.png"
path_list = [im_path for im_path in glob.glob(pngPath)]
path_list_parsed = [re.split('\\\\|\.', path) for path in path_list]
path_list_parsed_valid = [x for x in path_list_parsed if x[-1] == 'png']
path_list_parsed_valid = sorted(path_list_parsed_valid, key=lambda x:int(x[-2]))
all_pngs = []
#print(path_list_parsed_valid)

depthCounter = len(path_list_parsed_valid)

for path in path_list_parsed_valid:
    s = "\\"
    path = [f for f in path if f != '']
    s = s.join(path)
    s = s[:-4] + '.png'
    maskArray = np.array(Image.open(s))
    
    for i in range(len(COLOR_DICT)):
        
        index = np.where(np.all(maskArray == COLOR_DICT[i], axis = 2))
        zCor = np.zeros(len(index[0])) + depthCounter
        newCors = np.dstack((index[0], index[1], zCor))[0]
        organData[i] = np.concatenate((organData[i], (newCors)))
    
    depthCounter -= 1

for i in range(len(COLOR_DICT)):
    organColor[i] = np.zeros(organData[i].shape) + COLOR_DICT[i]

In [4]:
allOrganPts = np.concatenate(organData)
allOrganCol = np.concatenate(organColor)

In [5]:
displayPoints(allOrganPts[1::], allOrganCol[1::], 0.5)